# Supervised Learning 

- KNN model, RF model, SVC model Pipeline notebook

- KNN model pipeline module is knn_result function

- RF model pipeline module is randomforest_result function

- SVC model pipeline module is svc_result function

- There are four versions for pipelines: 

1-1. pipeline_scaling : Standard Scaling on X + Using Theoretical group variable

1-2. pipeline : Using original X + Using Theoretical group variable

1-3. pipeline(KMean ver.)_scaling:   Standard Scaling on X + Using KMean clustering group variable

1-4. pipeline(KMean ver.):   Using original X + Using KMean clustering group variable

- Output

1. Trainded Classifiers are all stored in the folders in result folders named ft_None_{}{}.format(threshold_value for PCA, if X is scaled or not)

undefined. CV results for train=validation dataset for all model options are stored in the same folder explained above as df_{}_{}.format(model_type, if it uses KMeans cluster group variable or not)

undefined. Classification results for the final test dataset of all model options are are stored in the same folder explained above as df_{}_creport_{}.format(model_type, if it uses KMeans cluster group variable or not)

In [1]:
# !pip install nbconvert 
# !jupyter nbconvert model_team14.ipynb --to script
#!pip install numpy -U

In [2]:
import model_team14 
from model_team14 import *

import os
import pandas as pd
import numpy as np
import pickle

from sklearn.model_selection import train_test_split, KFold, TimeSeriesSplit
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report
from sklearn.preprocessing import StandardScaler

/shared-libs/python3.9/py/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
def classification_report_csv(report):
    report_data = []
    lines = report.split('\n')
    
    for line in lines[2:len(lines)-5]:
        row = {}
        row_data = [val for val in line.split(' ') if val!='']
        row['class'] = round(float(row_data[0]),0)
        row['precision'] = float(row_data[1])
        row['recall'] = float(row_data[2])
        row['f1_score'] = float(row_data[3])
        row['support'] = float(row_data[4])
        row['accuracy']=float([val for val in lines[-4].split(' ') if val!=''][-2])
        report_data.append(row)
        
    df = pd.DataFrame(report_data)
    return df

In [4]:
############ to use kmean group, select_features-->select_features2 / modelname to add _km  ################# 

metadata=pd.read_csv('../data/full_info.csv')
threshold=0.3
criteria=None  

#############scaling###############################
path='../result/ft_{}_{}_scaling'.format(str(criteria), str(threshold))       

if not os.path.exists(path):
    os.mkdir(path)

## filtering criteria is not cumulative explained variance ratio but just explained variance rati

In [5]:
def knn_result(metadata, threshold, criteria ,param_knn, y_type='y_agg' ,cv='tss', test_year=5, n_splits=5, dtype='tr'): 

    if dtype=='tr':
        X=pd.read_csv('../data/X_data_tr.csv', index_col='date', parse_dates=True)
        y=pd.read_csv('../data/y_data_tr.csv', index_col='date', parse_dates=True)
    else:
        X=pd.read_csv('../data/X_data.csv', index_col='date', parse_dates=True)
        y=pd.read_csv('../data/y_data.csv', index_col='date', parse_dates=True)

################# revise
    df_feature=select_features(metadata, X, threshold, criteria=criteria)
    selected_features=list(df_feature[df_feature.select==1]['variable'])
    
    print(selected_features)
    
    ## train and validation set: X_train, y_train / final test set: X_test, y_test
    X_train=X[selected_features][:-(test_year*12)]  
    y_train=y[y_type][:-(test_year*12)]           
    X_test=X[selected_features][-(test_year*12):]   
    y_test=y[y_type][-(test_year*12):]
    
    ############################scaling#####################################
    sc = StandardScaler()
    X_scaled_train = sc.fit_transform(X_train)
    X_scaled_test=sc.transform(X_test)

    data_list=[X_train, X_test, X_scaled_train, X_scaled_test]
    name_list=['X_train', 'X_test', 'X_scaled_train', 'X_scaled_test']

    for x, name in zip(data_list, name_list):
        with open(path+'/{}.csv'.format(name),'wb') as f:
            pickle.dump(x, f)
    ###############################################################################
    
    ## cross validation for parameter tuning & training
    if cv=='block':
        split=KFold(n_splits=n_splits, shuffle=False)  
    else:
        split=TimeSeriesSplit(n_splits=n_splits)
        
    clf = GridSearchCV(KNeighborsClassifier(), param_knn, cv=split,  
                       verbose=3, n_jobs=-1, scoring=['recall_macro'],
                    refit='recall_macro'
                    )

    
    ############################scaling####################################
    clf.fit(X_scaled_train, y_train)
    
    ## training result
    ############################scaling####################################
    y_pred_prob=clf.predict_proba(X_scaled_test)
    y_pred = clf.predict(X_scaled_test)

    clf_report=classification_report(y_test, y_pred)

    param=clf.cv_results_['params']
    mean_test_score=clf.cv_results_['mean_test_recall_macro']
    std_test_score=clf.cv_results_['std_test_recall_macro']
    rank_test_score=clf.cv_results_['rank_test_recall_macro']

    for idx, x in enumerate(param):
        x['model']='KNN'
        x['data']=dtype
        x['y']=y_type
        x['cv']=cv
        x['mean_test_recall']=mean_test_score[idx] 
        x['std_test_recall']=std_test_score[idx]         
        x['rank_test_recall']=rank_test_score[idx]

    df_cvresult=pd.DataFrame(param)

    ################# revise
    with open (path+'/clf_knn_{}_{}_{}_t{}_spl{}.pkl'.format(y_type, dtype, cv, test_year, n_splits), 'wb') as f:  
        pickle.dump([clf, df_cvresult, y_pred, y_pred_prob, clf_report], f)


In [6]:
#df_feature=df_feature
param_knns=[{'n_neighbors':[1,2,3,5,10],
            'weights':['uniform','distance']}]  ## you can add more dictionary for other combinations of parameters.
y_types=['y_agg','y_oecd']
cvs=['tss']
test_year=5
n_splits=5
dtypes=['tr','ntr']

for param_knn in param_knns:
    for y_type in y_types:
        for cv in cvs:
            for dtype in dtypes:
                knn_result(metadata, threshold, criteria, param_knn, y_type=y_type ,cv=cv, test_year=test_year, n_splits=n_splits, dtype=dtype)

['IPMANSICS', 'PAYEMS', 'HOUST', 'ISRATIOx', 'T10YFFM', 'WILL5000INDFC', 'S&P PE ratio', 'BSI']
Fitting 5 folds for each of 10 candidates, totalling 50 fits
[CV 1/5] END n_neighbors=1, weights=uniform; recall_macro: (test=0.647) total time=   0.0s
[CV 2/5] END n_neighbors=1, weights=uniform; recall_macro: (test=0.651) total time=   0.0s
[CV 3/5] END n_neighbors=1, weights=uniform; recall_macro: (test=0.544) total time=   0.0s
[CV 4/5] END n_neighbors=1, weights=uniform; recall_macro: (test=0.673) total time=   0.0s
[CV 5/5] END n_neighbors=1, weights=uniform; recall_macro: (test=0.389) total time=   0.0s
[CV 1/5] END n_neighbors=1, weights=distance; recall_macro: (test=0.647) total time=   0.0s
/shared-libs/python3.9/py/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(

In [7]:
dict_knn={}

for param_knn in param_knns:
    for y_type in y_types:
        for cv in cvs:
            for dtype in dtypes:
################# revise
                with open (path+'/clf_knn_{}_{}_{}_t{}_spl{}.pkl'.format(y_type, dtype, cv, test_year, n_splits), 'rb') as f:
                    [clf, df_cvresult, y_pred, y_pred_prob, clf_report]=pickle.load(f)
################# revise
                    dict_knn['clf_knn_{}_{}_{}_t{}_spl{}'.format(y_type, dtype, cv, test_year, n_splits)]=[clf, df_cvresult, y_pred, y_pred_prob, clf_report]
                    
for idx, model in enumerate(dict_knn):
    if idx==0:
        df_knn=dict_knn[model][1]
    else:
        df_tmp=dict_knn[model][1]
        df_knn=pd.concat([df_knn, df_tmp])
                    
df_knn[df_knn.rank_test_recall==1]     
    

,n_neighbors,weights,model,data,y,cv,mean_test_recall,std_test_recall,rank_test_recall
8,10,uniform,KNN,tr,y_agg,tss,0.641336,0.168139,1
5,3,distance,KNN,ntr,y_agg,tss,0.495926,0.214698,1
8,10,uniform,KNN,tr,y_oecd,tss,0.703995,0.128376,1
8,10,uniform,KNN,ntr,y_oecd,tss,0.616371,0.120258,1


In [8]:
for idx, model in enumerate(dict_knn.keys()):
    
    report=dict_knn[model][-1]
    
    if idx==0:
        df_knn_creport=classification_report_csv(report)
        df_knn_creport['model']=model
    else:
        df_tmp=classification_report_csv(report)
        df_tmp['model']=model
        df_knn_creport=pd.concat([df_knn_creport, df_tmp])
        
df_knn_creport  ##[df_knn_creport_km['class']>0]

,class,precision,recall,f1_score,support,accuracy,model
0,0.0,0.96,0.89,0.93,56.0,0.87,clf_knn_y_agg_tr_tss_t5_spl5
1,1.0,0.00,0.00,0.00,2.0,0.87,clf_knn_y_agg_tr_tss_t5_spl5
2,2.0,0.67,1.00,0.80,2.0,0.87,clf_knn_y_agg_tr_tss_t5_spl5
0,0.0,0.98,0.96,0.97,56.0,0.92,clf_knn_y_agg_ntr_tss_t5_spl5
1,1.0,0.20,0.50,0.29,2.0,0.92,clf_knn_y_agg_ntr_tss_t5_spl5
2,2.0,0.00,0.00,0.00,2.0,0.92,clf_knn_y_agg_ntr_tss_t5_spl5
0,0.0,0.96,0.89,0.93,56.0,0.87,clf_knn_y_oecd_tr_tss_t5_spl5
1,1.0,0.25,0.50,0.33,4.0,0.87,clf_knn_y_oecd_tr_tss_t5_spl5
0,0.0,0.98,0.91,0.94,56.0,0.90,clf_knn_y_oecd_ntr_tss_t5_spl5
1,1.0,0.38,0.75,0.50,4.0,0.90,clf_knn_y_oecd_ntr_tss_t5_spl5


In [9]:
def randomforest_result(metadata, threshold, criteria, param_rf, y_type='y_agg' ,cv='tss', test_year=5, n_splits=5, dtype='tr'):
    
    if dtype=='tr':
        X=pd.read_csv('../data/X_data_tr.csv', index_col='date', parse_dates=True)
        y=pd.read_csv('../data/y_data_tr.csv', index_col='date', parse_dates=True)
    else:
        X=pd.read_csv('../data/X_data.csv', index_col='date', parse_dates=True)
        y=pd.read_csv('../data/y_data.csv', index_col='date', parse_dates=True)

################# revise
    df_feature=select_features(metadata, X, threshold, criteria=criteria) 
    selected_features=list(df_feature[df_feature.select==1]['variable'])
    
    print(selected_features)

    
    ## train and validation set: X_train, y_train / final test set: X_test, y_test
    X_train=X[selected_features][:-(test_year*12)]  
    y_train=y[y_type][:-(test_year*12)]           
    X_test=X[selected_features][-(test_year*12):]   
    y_test=y[y_type][-(test_year*12):]

    ############################scaling#####################################
    sc = StandardScaler()
    X_scaled_train = sc.fit_transform(X_train)
    X_scaled_test=sc.transform(X_test)
    
#     X_scaled_train=pd.DataFrame(X_scaled_train, columns=selected_features, index=X_train.index)
#     X_scaled_test=pd.DataFrame(X_scaled_test, columns=selected_features, index=X_test.index)
    
        
    ############################scaling#####################################
    
    ## cross validation for parameter tuning & training
    if cv=='block':
        split=KFold(n_splits=n_splits, shuffle=False)  
    else:
        split=TimeSeriesSplit(n_splits=n_splits)
        
    clf = GridSearchCV(RandomForestClassifier(n_jobs=-1, oob_score=True, random_state=14), 
                       param_rf, cv=split, 
                       verbose=3, n_jobs=-1, scoring=['recall_macro'],
                    refit='recall_macro'
                    )
    
    ############################scaling#####################################
    clf.fit(X_scaled_train, y_train)
    
    ## training result
    ############################scaling#####################################
    y_pred_prob=clf.predict_proba(X_scaled_test)
    y_pred = clf.predict(X_scaled_test)

    clf_report=classification_report(y_test, y_pred)

    param=clf.cv_results_['params']
    mean_test_score=clf.cv_results_['mean_test_recall_macro']
    std_test_score=clf.cv_results_['std_test_recall_macro']
    rank_test_score=clf.cv_results_['rank_test_recall_macro']

    for idx, x in enumerate(param):
        x['model']='RF'
        x['data']=dtype
        x['y']=y_type
        x['cv']=cv
        x['mean_test_recall']=mean_test_score[idx] 
        x['std_test_recall']=std_test_score[idx]         
        x['rank_test_recall']=rank_test_score[idx]

    df_cvresult=pd.DataFrame(param)

################# revise
# uncomment to save the results
    with open (path+'/clf_rf_{}_{}_{}_t{}_spl{}.pkl'.format(y_type, dtype, cv, test_year, n_splits), 'wb') as f: ################
        pickle.dump([clf, df_cvresult, y_pred, y_pred_prob, clf_report], f)


In [11]:
param_rfs=[{'n_estimators':[30,50,100],
             'max_features':[0.2, 0.3, 0.5, 0.7, 0.9],
           'class_weight':['balanced','balanced_subsample',None],
           'warm_start':[True,False]}]  ## you can add more dictionary for other combinations of parameters.
y_types=['y_agg','y_oecd']
cvs=['tss']
test_year=5
n_splits=5
dtypes=['tr','ntr']

for param_rf in param_rfs:
    for y_type in y_types:
        for cv in cvs:
            for dtype in dtypes:
                randomforest_result(metadata, threshold, criteria, param_rf, y_type=y_type ,cv=cv, test_year=test_year, n_splits=n_splits, dtype=dtype)

In [12]:
dict_rf={}

for param_rf in param_rfs:
    for y_type in y_types:
        for cv in cvs:
            for dtype in dtypes:
################# revise
                with open (path+'/clf_rf_{}_{}_{}_t{}_spl{}.pkl'.format(y_type, dtype, cv, test_year, n_splits), 'rb') as f:
                    [clf, df_cvresult, y_pred, y_pred_prob, clf_report]=pickle.load(f)
################# revise
                    dict_rf['clf_rf_{}_{}_{}_t{}_spl{}'.format(y_type, dtype, cv, test_year, n_splits)]=[clf, df_cvresult, y_pred, y_pred_prob, clf_report]
                    
for idx, model in enumerate(dict_rf):
    if idx==0:
        df_rf=dict_rf[model][1]
    else:
        df_tmp=dict_rf[model][1]
        df_rf=pd.concat([df_rf, df_tmp])
                    
df_rf[df_rf.rank_test_recall==1]    

In [ ]:
for idx, model in enumerate(dict_rf.keys()):
    
    report=dict_rf[model][-1]
    
    if idx==0:
        df_rf_creport=classification_report_csv(report)
        df_rf_creport['model']=model
    else:
        df_tmp=classification_report_csv(report)
        df_tmp['model']=model
        df_rf_creport=pd.concat([df_rf_creport, df_tmp])
        

df_rf_creport

In [ ]:
def svc_result(metadata, threshold, criteria, param_svc, y_type='y_agg' ,cv='tss', test_year=5, n_splits=5, dtype='tr'):
    
    if dtype=='tr':
        X=pd.read_csv('../data/X_data_tr.csv', index_col='date', parse_dates=True)
        y=pd.read_csv('../data/y_data_tr.csv', index_col='date', parse_dates=True)
    else:
        X=pd.read_csv('../data/X_data.csv', index_col='date', parse_dates=True)
        y=pd.read_csv('../data/y_data.csv', index_col='date', parse_dates=True)

################# revise
    df_feature=select_features(metadata, X, threshold, criteria=criteria)  #####################################
    selected_features=list(df_feature[df_feature.select==1]['variable'])
    
    print(selected_features)

    
    ## train and validation set: X_train, y_train / final test set: X_test, y_test
    X_train=X[selected_features][:-(test_year*12)]  
    y_train=y[y_type][:-(test_year*12)]           
    X_test=X[selected_features][-(test_year*12):]   
    y_test=y[y_type][-(test_year*12):]

    ############################scaling#####################################
    sc = StandardScaler()
    X_scaled_train = sc.fit_transform(X_train)
    X_scaled_test=sc.transform(X_test)
    ############################scaling#####################################
    
    ## cross validation for parameter tuning & training
    if cv=='block':
        split=KFold(n_splits=n_splits, shuffle=False)  
    else:
        split=TimeSeriesSplit(n_splits=n_splits)
    
    
    if y_type=='y_agg':
        cw_dict={0:1,1:5,2:5}
    else:
        cw_dict={0:1,1:5}
        
    
    clf = GridSearchCV(SVC(random_state=14, class_weight=cw_dict, probability=True), 
                       param_svc, cv=split, 
                       verbose=3, n_jobs=-1, scoring=['recall_macro'],
                    refit='recall_macro'
                    )
    
    ############################scaling#####################################
    clf.fit(X_scaled_train, y_train)
    
    ## training result
    ############################scaling#####################################
    y_pred_prob=clf.predict_proba(X_scaled_test)
    y_pred = clf.predict(X_scaled_test)

    clf_report=classification_report(y_test, y_pred)

    param=clf.cv_results_['params']
    mean_test_score=clf.cv_results_['mean_test_recall_macro']
    std_test_score=clf.cv_results_['std_test_recall_macro']
    rank_test_score=clf.cv_results_['rank_test_recall_macro']

    for idx, x in enumerate(param):
        x['model']='SVC'
        x['data']=dtype
        x['y']=y_type
        x['cv']=cv
        x['mean_test_recall']=mean_test_score[idx] 
        x['std_test_recall']=std_test_score[idx]         
        x['rank_test_recall']=rank_test_score[idx]

    df_cvresult=pd.DataFrame(param)
################# revise
# uncomment to save the results
    with open (path+'/clf_svc_{}_{}_{}_t{}_spl{}.pkl'.format(y_type, dtype, cv, test_year, n_splits), 'wb') as f:
        pickle.dump([clf, df_cvresult, y_pred, y_pred_prob, clf_report], f)


In [0]:
param_svcs=[{'C':[0.5, 1, 2, 5],
             'kernel':['poly','rbf','sigmoid']}]  ## you can add more dictionary for other combinations of parameters.
y_types=['y_agg','y_oecd']
cvs=['tss']
test_year=5
n_splits=5
dtypes=['tr','ntr']

for param_svc in param_svcs:
    for y_type in y_types:
        for cv in cvs:
            for dtype in dtypes:
                svc_result(metadata, threshold, criteria, param_svc, y_type=y_type ,cv=cv, test_year=test_year, n_splits=n_splits, dtype=dtype)

In [ ]:
dict_svc={}

for param_svc in param_svcs:
    for y_type in y_types:
        for cv in cvs:
            for dtype in dtypes:
################# revise
                with open (path+'/clf_svc_{}_{}_{}_t{}_spl{}.pkl'.format(y_type, dtype, cv, test_year, n_splits), 'rb') as f:
                    [clf, df_cvresult, y_pred, y_pred_prob, clf_report]=pickle.load(f)
################# revise
                    dict_svc['clf_svc_{}_{}_{}_t{}_spl{}'.format(y_type, dtype, cv, test_year, n_splits)]=[clf, df_cvresult, y_pred, y_pred_prob, clf_report]
                    
for idx, model in enumerate(dict_svc):
    if idx==0:
        df_svc=dict_svc[model][1]
    else:
        df_tmp=dict_svc[model][1]
        df_svc=pd.concat([df_svc, df_tmp])
                    
df_svc[df_svc.rank_test_recall==1]    

In [ ]:
for idx, model in enumerate(dict_svc.keys()):
    
    report=dict_svc[model][-1]
    
    if idx==0:
        df_svc_creport=classification_report_csv(report)
        df_svc_creport['model']=model
    else:
        df_tmp=classification_report_csv(report)
        df_tmp['model']=model
        df_svc_creport=pd.concat([df_svc_creport, df_tmp])
        

df_svc_creport

In [ ]:
dict_list=[df_knn, df_knn_creport, df_rf, df_rf_creport, df_svc, df_svc_creport]
id_list=['df_knn', 'df_knn_creport', 'df_rf', 'df_rf_creport', 'df_svc', 'df_svc_creport']

for i, dict_ in enumerate(dict_list):
################# revise
    with open(path+'/{}.pkl'.format(id_list[i]), 'wb') as f:               
        pickle.dump(dict_, f)

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=dc2156ff-f31b-485a-9893-d89a520307c4' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>